In [2]:
import os
import hashlib

In [5]:
import mkgu

In [1]:
import dataset

ModuleNotFoundError: No module named 'dataset'

In [3]:
def hash(path, buffer_size=64*2**10):
    sha1 = hashlib.sha1()
    with open(path, "rb") as f:
        buffer = f.read(buffer_size)
        while len(buffer) > 0:
            sha1.update(buffer)
            buffer = f.read(buffer_size)
    return sha1.hexdigest()

In [20]:
class KnownFile(object):
    def __init__(self, path, persister=None):
        self.path = path
        self.exists = os.path.exists(path)
        if self.exists:
            self.realpath = os.path.realpath(path)
            self.isfile = os.path.isfile(self.realpath)
            if self.isfile:
                self.sha1 = hash(self.realpath)
                if persister is not None:
                    self.persister = persister
                    persister.persist(self)
                    
    def __repr__(self):
        return str((self.__class__.__name__, vars(self)))

In [14]:
test_dir = "/braintree/data2/active/common/for_jjpr"

In [15]:
test_not_exists = "/braintree/data2/active/common/for_jjpr/foobar"

In [16]:
xls_files = [
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-000936/5602-2 SN1025-000936.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5E.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5F.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5H.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-5-SN0656.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2466-1-SN0658.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/090609B-4F.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/090609B-8E.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/2467-6_SN0649.xls"    
]

In [17]:
# Tito IT A is 657 per /braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages
tito_it_a_xls_file = "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls"

In [21]:
kf_xls = KnownFile(tito_it_a_xls_file)

In [22]:
kf_xls

('KnownFile', {'path': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls', 'exists': True, 'realpath': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls', 'isfile': True, 'sha1': '7d705186a726776e9517269dafd01d6407d0ce2e'})

In [11]:
def persister_setup(persister_class):
    db = persister_db()
    return persister_class(db)

In [12]:
def persister_db(db_file_path=None): 
    if db_file_path is None:
        db_file_path = os.path.join(os.path.dirname(mkgu.__file__), "knownfile.db")
    db_url = "sqlite:///" + db_file_path
    db = dataset.connect(db_url)
    return db

In [8]:
class Persister(object):
    def __init__(self, db):
        self.db = db
 
    def persist(self):
        raise NotImplementedError()

    def keyed_upsert(self, tbl_name, keys, data):
        tbl = self.db[tbl_name]
        record_id = tbl.upsert(data, keys=keys)
        if type(record_id) == bool:
            record_id = tbl.find_one(**{key: data[key] for key in keys})["id"]
        return record_id                

In [9]:
class KnownFilePersister(Persister):
    def persist(self, knownfile):
        data = {
            "realpath": knownfile.realpath,
            "sha1": knownfile.sha1
        }
        self.id = self.keyed_upsert("file", ["sha1"], data)